# Finding relevant code changes in frameworks and packages

The goal of this notebook is to track evolving code bases by first extracting changes made via the git log. These can then be filtered for the correct timeframe and relevant functions. The next step would then be to analyse the changes and decide whether they are relevant to a developer that uses that part of the code for differential testing or not.

## Imports

In [ ]:
import os

## Setup

Paste the github giturl of the library you would like to analyse in the following cell:

In [ ]:
git_url = "https://github.com/pytorch/pytorch.git"

## Create a bare clone of the library, which only includes repository data

In this way, we do not have to download the code, but still get access to the commit log.

In [ ]:
# create a temporary directory for a bare clone of a give library
try:
    os.mkdir('temp_bare_clone')
except:
    pass

In [ ]:
# create the bare clone
!git clone --bare {git_url} temp_bare_clone

In [ ]:
%cd temp_bare_clone

In [ ]:
!git log HEAD